In [20]:
# Libraries to help with reading and manipulating data
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.utils import resample
import pandas as pd
import numpy as np

from datetime import datetime

# Libaries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns

# To tune model, get different metric scores, and split data
from sklearn.metrics import (
    f1_score,
    accuracy_score,
    recall_score,
    precision_score,
    confusion_matrix,
    roc_auc_score,
)
from sklearn.metrics import ConfusionMatrixDisplay

from sklearn import metrics

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score

# To be used for data scaling and one hot encoding
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

# To impute missing values
from sklearn.impute import SimpleImputer

# To oversample and undersample data
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

# To do hyperparameter tuning
from sklearn.model_selection import RandomizedSearchCV

# To be used for creating pipelines and personalizing them
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# To define maximum number of columns to be displayed in a dataframe
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# To supress scientific notations for a dataframe
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To help with model building
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    AdaBoostClassifier,
    GradientBoostingClassifier,
    RandomForestClassifier,
    BaggingClassifier,
)
from xgboost import XGBClassifier

# To suppress scientific notations
pd.set_option("display.float_format", lambda x: "%.3f" % x)

# To suppress warnings
import warnings
warnings.filterwarnings("ignore")

In [4]:
train_data = pd.read_csv('train_waves.csv') ##  read the data

test_data = pd.read_csv('test_waves.csv') ##  read the data


In [5]:
train_data.head()


,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m)
0,2017424150,2017,4,24,1,50,23,3.000,3.600,4.170,3.330,349,1019.100,4.700,5.100,4.200,0.390
1,2017424250,2017,4,24,2,50,30,2.900,3.300,5.260,4.210,358,1018.800,4.600,4.800,4.100,0.650
2,2017424350,2017,4,24,3,50,360,2.200,2.500,5.880,4.790,11,1018.300,4.600,4.700,4.200,0.900
3,2017424450,2017,4,24,4,50,355,2.500,3.100,5.880,4.950,23,1018.100,4.600,4.600,4.200,0.910
4,2017424550,2017,4,24,5,50,355,3.500,4.500,6.250,4.980,26,1018.300,4.500,4.600,4.000,0.880


In [6]:
df=train_data.copy()


In [7]:
# Calculate the sine and cosine of wind direction
df['WDIR_sin'] = np.sin(np.radians(df['WDIR(degT)']))
df['WDIR_cos'] = np.cos(np.radians(df['WDIR(degT)']))
#wind speed cubed
df['WSCB']=np.power(df['WSPD(m/s)'],3)
#dominant wind power squared
df['DPD2']=np.power(df['DPD(sec)'],2)
#temperature differene between air and water
df['TDIF']=df['ATMP(degC)']-df['WTMP(degC)']
test_data['WDIR_sin'] = np.sin(np.radians(test_data['WDIR(degT)']))
test_data['WDIR_cos'] = np.cos(np.radians(test_data['WDIR(degT)']))
test_data['WSCB']=np.power(test_data['WSPD(m/s)'],3)
test_data['DPD2']=np.power(test_data['DPD(sec)'],2)
test_data['TDIF']=test_data['ATMP(degC)']-test_data['WTMP(degC)']

calculating the sine and cosine: Instead of using the raw wind direction feature, it may be helpful to convert it into its sine and cosine components to capture any directional patterns in the wind that affect wave height.

In [8]:
df.head()


,ID,#YY,MM,DD,hh,mm,WDIR(degT),WSPD(m/s),GST(m/s),DPD(sec),APD(sec),MWD(degT),PRES(hPa),ATMP(degC),WTMP(degC),DEWP(degC),WVHT(m),WDIR_sin,WDIR_cos,WSCB,DPD2,TDIF
0,2017424150,2017,4,24,1,50,23,3.000,3.600,4.170,3.330,349,1019.100,4.700,5.100,4.200,0.390,0.391,0.921,27.000,17.389,-0.400
1,2017424250,2017,4,24,2,50,30,2.900,3.300,5.260,4.210,358,1018.800,4.600,4.800,4.100,0.650,0.500,0.866,24.389,27.668,-0.200
2,2017424350,2017,4,24,3,50,360,2.200,2.500,5.880,4.790,11,1018.300,4.600,4.700,4.200,0.900,-0.000,1.000,10.648,34.574,-0.100
3,2017424450,2017,4,24,4,50,355,2.500,3.100,5.880,4.950,23,1018.100,4.600,4.600,4.200,0.910,-0.087,0.996,15.625,34.574,0.000
4,2017424550,2017,4,24,5,50,355,3.500,4.500,6.250,4.980,26,1018.300,4.500,4.600,4.000,0.880,-0.087,0.996,42.875,39.062,-0.100


In [9]:
df.shape


(3886, 22)

In [10]:
df.columns


Index(['ID', '#YY', 'MM', 'DD', 'hh', 'mm', 'WDIR(degT)', 'WSPD(m/s)',
       'GST(m/s)', 'DPD(sec)', 'APD(sec)', 'MWD(degT)', 'PRES(hPa)',
       'ATMP(degC)', 'WTMP(degC)', 'DEWP(degC)', 'WVHT(m)', 'WDIR_sin',
       'WDIR_cos', 'WSCB', 'DPD2', 'TDIF'],
      dtype='object')

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
y=df['WVHT(m)']
df=df.drop(columns=['WVHT(m)'])
X = df.iloc[:, [2, 4, *range(6, df.shape[1])]]
X=sc.fit_transform(X)

In [12]:
from sklearn.svm import SVR
model=SVR()
model.fit(X,y)

SVR()

In [13]:
X_test=test_data.iloc[:, [2, 4, *range(6, df.shape[1])]]
X_test=sc.transform(X_test)
#X_test.head()

In [14]:
pred=model.predict(X_test)


In [16]:
pred

array([0.25242986, 0.39072346, 0.36462452, ..., 0.72778994, 1.07483464,
       1.02982388])

In [17]:
output=pd.DataFrame({'Id':test_data.ID,'WVHT(m)':pred})


In [18]:
output.head()

,Id,WVHT(m)
0,2019722240,0.252
1,2019722340,0.391
2,201973140,0.365
3,2019761040,0.276
4,2019761140,0.300


In [24]:
rmse = np.sqrt(mean_squared_error(y[:1535], pred))
print("{}".format( rmse))

0.5367324341121634
